In [14]:
import pandas as pd
import holidays
import numpy as np

from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error,mean_squared_error

from statsforecast.models import AutoARIMA


In [15]:
df=pd.read_csv("/home/wapun/Documents/demand/arimax/TDinprogress.csv")
df.rename({'Unnamed: 0':'Time'},axis=1,inplace=True)
df['Time']=pd.to_datetime(df.Time,utc=True)
df.set_index('Time',inplace=True,drop=True)
df.index=df.index.tz_convert('Europe/Warsaw')
df


,PSEforecast,Load,holiday,TD,TG,"TD[6, 0, 1]"
Time,,,,,,
2015-01-10 00:00:00+01:00,17000.0,17109.375,False,-2279.250,-48.125,NaN
2015-01-10 01:00:00+01:00,16200.0,16438.150,False,-2234.775,-46.600,NaN
2015-01-10 02:00:00+01:00,15800.0,16122.300,False,-2148.525,-8.800,NaN
2015-01-10 03:00:00+01:00,15700.0,15987.900,False,-2082.875,26.000,NaN
2015-01-10 04:00:00+01:00,15700.0,15951.900,False,-2266.525,-220.775,NaN
...,...,...,...,...,...,...
2024-02-28 17:00:00+01:00,23557.0,23578.163,False,576.713,-110.000,NaN
2024-02-28 18:00:00+01:00,23558.0,23746.113,False,507.601,242.325,NaN
2024-02-28 19:00:00+01:00,23408.0,23638.600,False,518.563,95.812,NaN


In [3]:
holidaysPL=holidays.PL()
df['date'] = df.index.date
df['holiday']=df.date.transform(lambda x: x in holidaysPL)
df.drop('date',axis=1,inplace=True)


In [4]:
np.unique(df.loc[(df.holiday==True)&(df.index.dayofweek==5)].index.date)

array([datetime.date(2015, 8, 15), datetime.date(2015, 12, 26),
       datetime.date(2017, 11, 11), datetime.date(2018, 1, 6),
       datetime.date(2020, 8, 15), datetime.date(2020, 12, 26),
       datetime.date(2021, 5, 1), datetime.date(2021, 12, 25),
       datetime.date(2022, 1, 1), datetime.date(2023, 11, 11),
       datetime.date(2024, 1, 6)], dtype=object)

In [5]:
def dataSplit(dataframe,end_train):
    day=pd.to_timedelta(1,'d')
    year=pd.Timedelta(365,'d')
    train_data=dataframe.loc[dataframe.index<end_train]
    test_data=dataframe.loc[(dataframe.index>=end_train)&(dataframe.index.date<end_train.date()+day+day)]
    return train_data,test_data


In [18]:
end_train=pd.to_datetime('2022-02-28 10:00',utc=True).tz_convert('Europe/Warsaw')
day=pd.to_timedelta(1,'day')

trainData,testData=dataSplit(df,end_train)





In [19]:
pVal=[0,1,2,3,4,6,8,10,16,24]
dVal=[0,1]
qVal=[0,1,2,3,4,6,8,10,16,24]
orders=[]
orderss=[[2,0,2],[2,0,10],[3,0,0],[3,0,1],[4,0,1],[4,0,7],[5,0,0],[6,0,1],[8,0,0],[8,0,1],[9,0,1],[10,0,0],[11,0,0],[12,0,1]]
testMAE=[]
testMSE=[]
testMAPE=[]
trainMAE=[]
trainMSE=[]
trainMAPE=[]

In [21]:
lol=orderss[0]
column='TD'
daye=pd.to_datetime('2022-02-28 10:00',utc=True).tz_convert('Europe/Warsaw')
df[column+str(lol)]=pd.NA
for date in pd.date_range(daye,daye+365*day):
    
    trainData,testData=dataSplit(df,date)
    model=SARIMAX(trainData[column],order=lol,freq='H')
    res=model.fit()
    predi=res.predict(testData.index[0],testData.tail(1).index[0])
    predi=predi[predi.index.date==date.date()+day]
    df.loc[df.index.date==(date.date()+day),column+str(lol)]=predi
    print(date)


/home/wapun/anaconda3/envs/datascience/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.90474D+00    |proj g|=  2.03818D-03

At iterate    5    f=  6.90473D+00    |proj g|=  6.48730D-04

At iterate   10    f=  6.90472D+00    |proj g|=  2.21361D-03

At iterate   15    f=  6.90470D+00    |proj g|=  8.21991D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     15     17      1     0     0   8.220D-06   6.905D+00
  F =   6.9047044546733352     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
2022-02-

/home/wapun/anaconda3/envs/datascience/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.90474D+00    |proj g|=  2.02564D-03


 This problem is unconstrained.


...

/home/wapun/anaconda3/envs/datascience/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.91028D+00    |proj g|=  2.01230D-03


 This problem is unconstrained.



At iterate    5    f=  6.91027D+00    |proj g|=  6.36771D-04

At iterate   10    f=  6.91025D+00    |proj g|=  2.14378D-03

At iterate   15    f=  6.91024D+00    |proj g|=  3.36131D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     15     18      1     0     0   3.361D-06   6.910D+00
  F =   6.9102407279851787     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
2022-04-19 11:00:00+02:00


/home/wapun/anaconda3/envs/datascience/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


In [11]:
df

,PSEforecast,Load,holiday,TD,TG,"TD[2, 0, 2]"
Time,,,,,,
2015-01-10 00:00:00+01:00,17000.0,17109.375,False,-2279.250,-48.125,NaN
2015-01-10 01:00:00+01:00,16200.0,16438.150,False,-2234.775,-46.600,NaN
2015-01-10 02:00:00+01:00,15800.0,16122.300,False,-2148.525,-8.800,NaN
2015-01-10 03:00:00+01:00,15700.0,15987.900,False,-2082.875,26.000,NaN
2015-01-10 04:00:00+01:00,15700.0,15951.900,False,-2266.525,-220.775,NaN
...,...,...,...,...,...,...
2024-02-28 17:00:00+01:00,23557.0,23578.163,False,576.713,-110.000,NaN
2024-02-28 18:00:00+01:00,23558.0,23746.113,False,507.601,242.325,NaN
2024-02-28 19:00:00+01:00,23408.0,23638.600,False,518.563,95.812,NaN


In [22]:

df[(df.index>(end_train))&(df.index<(end_train+2*day))]

,PSEforecast,Load,holiday,TD,TG,"TD[6, 0, 1]"
Time,,,,,,
2022-02-28 12:00:00+01:00,23350.0,23440.450,False,-269.150,167.487,NaN
2022-02-28 13:00:00+01:00,23650.0,23535.775,False,-544.088,98.937,NaN
2022-02-28 14:00:00+01:00,23700.0,23487.225,False,-83.037,568.813,NaN
2022-02-28 15:00:00+01:00,23900.0,23491.500,False,-190.125,41.125,NaN
2022-02-28 16:00:00+01:00,24250.0,23802.063,False,-377.550,201.450,NaN
2022-02-28 17:00:00+01:00,25250.0,24358.700,False,-254.075,-16.725,NaN
2022-02-28 18:00:00+01:00,25400.0,25054.600,False,-350.850,368.500,NaN
2022-02-28 19:00:00+01:00,25300.0,24941.938,False,-569.187,-3.612,NaN
2022-02-28 20:00:00+01:00,24800.0,24531.450,False,-367.051,226.299,NaN


In [24]:
df.to_csv('TD[6,0,1]')